This notebook assumes that you have a running instance of Neo4J Aura. You can create a free one by heading over to https://neo4j.com/

Once you have your instance, save its id as `neo4j_instance_id` and its password as `neo4j_password` as Colab secrets and grant this notebook access to said secrets.

In [1]:
!pip install neo4j yfiles_jupyter_graphs_for_neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.6 MB/s eta 0:00:00


In [2]:
from neo4j import GraphDatabase
from google.colab import userdata

neo4j_instance_id = userdata.get('neo4j_instance_id')
neo4j_password = userdata.get('neo4j_password')

driver = GraphDatabase.driver(f"neo4j+s://{neo4j_instance_id}.databases.neo4j.io:7687", auth=("neo4j", neo4j_password))

In [3]:
def creating_vertices(tx):
    query = """
      CREATE (a:Account {id: 'ACC123456', name: 'Walt Whitman', type: 'Checking', balance: 1200.00})
      CREATE (t:Transaction {id: '1234567890', amount: 150.75, type: 'Purchase', merchant: 'ABC Store', location: 'New York, NY', datetime: '2024-08-25 14:32'})

    RETURN *
    """
    tx.run(query)

# Run the query
with driver.session() as session:
    session.execute_write(creating_vertices)

In [4]:
def creating_edges(tx):
    query = """
      MATCH (a:Account {id: 'ACC123456'}), (t:Transaction {id: '1234567890'})
        CREATE (a)-[:MADE]->(t)

    RETURN *
    """
    tx.run(query)

with driver.session() as session:
    session.execute_write(creating_edges)

In [5]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

def show_graph(driver):
    query = """
    MATCH (n)-[r]->(m)
    RETURN n, r, m
    """
    g.show_cypher(query)

show_graph(driver)

GraphWidget(layout=Layout(height='500px', width='100%'))

In [7]:
def querying_data(driver):
    query = """
    MATCH (a:Account {id: 'ACC123456'})-[:MADE]->(t:Transaction)
    RETURN t
    """
    g.show_cypher(query)

querying_data(driver)

GraphWidget(layout=Layout(height='500px', width='100%'))

In [8]:
def filtering_data(driver):
    query = """
    MATCH (a:Account)-[r:MADE]->(t:Transaction)
    WHERE t.amount > 100
    RETURN a, r, t
    """
    g.show_cypher(query)

filtering_data(driver)

GraphWidget(layout=Layout(height='500px', width='100%'))

In [9]:
def aggregating_data(tx):
    query = """
      MATCH (a:Account {id: 'ACC123456'})-[:MADE]->(t:Transaction)
      RETURN a.id AS account_id, SUM(t.amount) AS total_spent
    """
    result = tx.run(query)
    return [record.data() for record in result]

with driver.session() as session:
    results = session.execute_read(aggregating_data)
    print(results)

[{'account_id': 'ACC123456', 'total_spent': 150.75}]


In [10]:
def upserting_vertices(tx):
    query = """
      MERGE (a:Account {id: 'ACC123456'})
      ON MATCH SET a.balance = 1300.00
      ON CREATE SET a.name = 'Walt Whitman', a.balance = 1200.00, a.type = 'Checking'

    RETURN *
    """
    tx.run(query)

with driver.session() as session:
    session.execute_write(upserting_vertices)

In [11]:
def distinct_data(tx):
    query = """
      MATCH (a:Account)-[:MADE]->(t:Transaction)
      RETURN DISTINCT t.merchant
    """
    result = tx.run(query)
    return [record.data() for record in result]

with driver.session() as session:
    results = session.execute_read(distinct_data)
    print(results)

[{'t.merchant': 'ABC Store'}]


In [12]:
def label_data_with_case(tx):
    query = """
      MATCH (a:Account)-[:MADE]->(t:Transaction)
      RETURN a.id,
        CASE
          WHEN t.amount > 1000 THEN 'High-value transaction'
          ELSE 'Regular transaction'
        END AS transaction_type
    """
    result = tx.run(query)
    return [record.data() for record in result]

with driver.session() as session:
    results = session.execute_read(label_data_with_case)
    print(results)

[{'a.id': 'ACC123456', 'transaction_type': 'Regular transaction'}]


In [13]:
def ordering_data(tx):
    query = """
      MATCH (a:Account)-[:MADE]->(t:Transaction)
      RETURN a.id, t.amount
      ORDER BY t.amount DESC
    """
    result = tx.run(query)
    return [record.data() for record in result]

with driver.session() as session:
    results = session.execute_read(ordering_data)
    print(results)

[{'a.id': 'ACC123456', 't.amount': 150.75}]


In [14]:
def deleting_vertices_and_edges(tx):
    query = """
    MATCH (a:Account {id: 'ACC123456'})-[r:MADE]->(t:Transaction)
    DELETE r, t, a
    """
    tx.run(query)

with driver.session() as session:
    session.execute_write(deleting_vertices_and_edges)

In [6]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()